In [1]:
import numpy as np
import pandas as pd
import warnings,gc,os,time,math
from datetime import datetime
warnings.filterwarnings('ignore')
import matplotlib as mp
from matplotlib import pyplot as plt
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

In [ ]:
filename='ud20201019'
num='_34' 
today='20201023'

datap='/home/jupyter/LQ/VW/data/'
new_datap='/home/jupyter/LCG/dazhong/V2/%s/'%(filename) #读取阿桂的数据
savep='/home/jupyter/LQ/VW/20191217_VW_下次保养里程预测/20201023自动化更新数据/%s%s/'%(today,str(num))

if os.path.exists(savep):
    print('True')
else:
    os.mkdir(savep)
    
store_num=num.replace('_','')
store_num=int(store_num)
print(store_num)

if (store_num<=20):
    baoyang_id_max=10
elif (store_num>20)&(store_num<=40):
    baoyang_id_max=11
elif (store_num>40)&(store_num<=50):
    baoyang_id_max=12
elif store_num>50:
    baoyang_id_max=13
    
print('biggest batch :',baoyang_id_max)

s1=time.time()

In [2]:
def check_df(df):
    df=df.drop_duplicates()
    print(df.shape)
    print(df[['VIN','修理日期']].drop_duplicates().shape)
    print(df['VIN'].nunique())
    
def check_run_time(s):
    r=(time.time()-s)/60
    return "Run Mins: %f"%r

def get_miss_cols(df):
    tt=pd.isnull(df).sum().reset_index()
    tt.columns=['字段名','缺失值']
    tt['缺失值占比']=tt['缺失值']/df.shape[0]
    tt=tt.sort_values(by=['缺失值占比'],ascending=False).reset_index(drop=True)
    miss_cols=tt[tt['缺失值占比']>0]['字段名'].unique().tolist()
    miss_df=tt[tt['缺失值占比']>0]
    return miss_cols,miss_df

def get_user(df):
    df['VIN']=df['VIN'].apply(str)
    df['车身型式']=df['VIN'].apply(lambda x:x[3])
    df['变速器']=df['VIN'].apply(lambda x:x[4])
    df['乘员保护系统']=df['VIN'].apply(lambda x:x[5])
    df['车辆等级']=df['VIN'].apply(lambda x:x[6:8])
    df['生产年份']=df['VIN'].apply(lambda x:x[9])
    df['装配厂']=df['VIN'].apply(lambda x:x[10])
    for col in ['车身型式', '变速器', '乘员保护系统', '车辆等级', '生产年份', '装配厂']:
        df[col]=df[col].astype(str)
    df['用户属性']=list(map(lambda a,b,c,d,e,f:a+','+b+','+c+','+d+','+e+','+f,df['车身型式'],df['变速器'],
                        df['乘员保护系统'],df['车辆等级'],df['生产年份'],df['装配厂']))
    return df

def get_last_run(df1):
    df1['修理日期']=pd.to_datetime(df1['修理日期'])
    df1['购车时间']=pd.to_datetime(df1['购车时间'])
    df1=df1.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df1['修理日期_last']=df1.groupby('VIN')['修理日期'].shift(-1)
    df1.loc[df1['修理日期_last'].isnull(),'修理日期_last']=df1['购车时间']
    df1['天数差']=(df1['修理日期']-df1['修理日期_last']).apply(lambda x:x.days)

    df1['公里数_last']=df1.groupby('VIN')['公里数'].shift(-1)
    df1.loc[df1['公里数_last'].isnull(),'公里数_last']=0
    df1['公里数差']=df1['公里数']-df1['公里数_last']
    
    df1['日均']=df1['公里数差']/(df1['天数差'])

    return df1

In [3]:
def load_data(df,veh):
    
    if '购车时间' in veh.columns.tolist():
        veh.rename(columns={'购车时间':'购车时间_in_veh'},inplace=True)
        
    same_cols=[x for x in list(veh.columns) if x in list(df.columns)]
    same_cols.remove('VIN')
    if len(same_cols)>0:
        print(same_cols)
        df=df.drop(same_cols,axis=1)

    dff=veh.merge(df,on=['VIN'],how='left')
    dff.loc[dff['购车时间'].isnull(),'购车时间']=dff['购车时间_in_veh']
    dff.loc[dff['修理日期'].isnull(),'修理日期']=dff['购车时间']
    dff.loc[dff['公里数'].isnull(),'公里数']=0
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    dff['修理日期']=pd.to_datetime(dff['修理日期'])
    check_df(dff)


    revise_types=['首保','常规保养','附件安装','一般维修','事故车维修','普通索赔','服务活动',
                  '二手车认证维修','内部车辆维修']
    for col in revise_types:
        dff[col]=dff[col].fillna(0)

    mis1,mis2=get_miss_cols(dff)
    print(mis2.head(50))

    return dff,veh2

In [5]:
df=pd.read_pickle(savep+'final_zhudan.pkl')
print(df['VIN'].nunique())
# print(df.columns.tolist())
# print(df.shape)
# print(df[['VIN','修理日期']].drop_duplicates().shape)

veh=pd.read_pickle(savep+'vehicle_info.pkl')
# print(list(veh.columns))
# print(veh.shape)
print(veh['VIN'].nunique())

2020-09-29
446090
466677


In [6]:
same=[x for x in df.columns.tolist() if x in veh.columns.tolist()]
print(same)

['VIN', 'brand', '保养状态', '在店状态', '常去经销商', '购车时间', '人群_一级分类', '人群_二级分类']


In [7]:
df=df.drop(['brand', '保养状态', '在店状态', '常去经销商','人群_一级分类', '人群_二级分类'],axis=1)
veh.loc[veh['新购车时间'].isnull(),'新购车时间']=veh[veh['新购车时间'].isnull()]['购车时间']
veh=veh.drop(['购车时间'],axis=1)

In [8]:
dff=veh.merge(df,on=['VIN'],how='left')
print(dff['VIN'].nunique())

466677


In [9]:
dff.loc[dff['购车时间'].isnull(),'购车时间']=dff[dff['购车时间'].isnull()]['新购车时间']

In [10]:
dff.loc[dff['修理日期'].isnull(),'修理日期']=dff[dff['修理日期'].isnull()]['购车时间']
dff.loc[dff['公里数'].isnull(),'公里数']=0

In [11]:
check_df(dff)

(1505938, 62)
(1505938, 2)
466677


In [12]:
check=dff[dff['购车时间']>dff['修理日期']]
print(len(check))

0


In [13]:
dff=dff[dff['购车时间']<=dff['修理日期']]
dff['车龄bin']=(pd.to_datetime(today)-dff['购车时间']).apply(lambda x:math.ceil(x.days/180))

revise_types=['首保','常规保养','附件安装','一般维修','事故车维修','普通索赔','服务活动',
                '二手车认证维修','内部车辆维修']
for col in revise_types:
    dff[col]=dff[col].fillna(0)

In [14]:
veh2=dff[['VIN','一级车系','二级车系','常去经销商','购车时间', '车型档次','在店状态', '保养状态', 
          '人群_一级分类','人群_二级分类','使用性质','用户属性']].drop_duplicates().reset_index(drop=True)
print(veh2.shape)
print(veh2['VIN'].nunique())
print(pd.isnull(veh2).sum())

(466677, 12)
466677
VIN            0
一级车系           0
二级车系           0
常去经销商      20357
购车时间           0
车型档次           0
在店状态           0
保养状态           0
人群_一级分类        0
人群_二级分类        0
使用性质           0
用户属性           0
dtype: int64


In [15]:
def fenlei(dff):

    dff=dff.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    dff['修理日期']=pd.to_datetime(dff['修理日期'])
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    
    if '常规次数' in list(dff.columns):
        del dff['常规次数']
    tmp=dff[dff['首保']!=1]
    tmp=tmp[tmp['常规保养']==1]
    tmpp=tmp.groupby('VIN')['修理日期'].nunique().reset_index(name='常规次数')
    dff=dff.merge(tmpp,on=['VIN'],how='left')
    
    if '首保次数' in list(dff.columns):
        del dff['首保次数']
    tmp=dff[dff['首保']==1]
    tmp['首保次数']=1
    dff=dff.merge(tmp[['VIN','首保次数']].drop_duplicates(),on=['VIN'],how='left')

    for col in ['首保次数','常规次数']:
        dff[col]=dff[col].fillna(0)

    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==0),'人群']='有首保,无常规,预测第1次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==1),'人群']='有首保,1常规,预测第2次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==2),'人群']='有首保,2次常规,预测第3次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']>=3),'人群']='3次及以上常规,预测接下去常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==1),'人群']='无首保,1次常规,预测第2次常规(购车)'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==2),'人群']='无首保,2次常规,预测第3次常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']>=3),'人群']='3次及以上常规,预测接下去常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==0)&(dff['车龄bin']<=3),'人群']='无保养记录,预测首保'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==0)&(dff['车龄bin']>3),'人群']='无保养习惯'

    print('查看是否有没有被分配到的人群')
    c=dff[['VIN','人群','车龄bin']].drop_duplicates().reset_index(drop=True)
    print(c.shape)
    print(c['VIN'].nunique())
    print(pd.isnull(c).sum())
    
    print('查看对应的每一类人群的VIN数量')
    tmp=dff.groupby('人群')['VIN'].nunique().reset_index(name='人数')
    print(tmp.head(10))
    print(tmp['人数'].sum(0))
    print(dff['VIN'].nunique())
    
    print('############最终要的是下面的二个人群###############')
    print('拆分有保养与没有保养的人群')
    baoyangdf1=dff[(dff['首保']==1)|(dff['常规保养']==1)]
    print('有保养',baoyangdf1['人群'].unique().tolist())
    
    nobaoyang1=dff[~(dff['VIN'].isin(set(baoyangdf1['VIN'])))]
    print('没有保养',nobaoyang1['人群'].unique().tolist())
    
    return baoyangdf1,nobaoyang1

In [16]:
baoyangdf1,nobaoyang1=fenlei(dff)

查看是否有没有被分配到的人群
(466677, 3)
466677
VIN      0
人群       0
车龄bin    0
dtype: int64
查看对应的每一类人群的VIN数量
                     人群      人数
0       3次及以上常规,预测接下去常规  161005
1                 无保养习惯   32268
2            无保养记录,预测首保   19252
3  无首保,1次常规,预测第2次常规(购车)   41856
4      无首保,2次常规,预测第3次常规    7145
5       有首保,1常规,预测第2次常规   47602
6      有首保,2次常规,预测第3次常规   38537
7       有首保,无常规,预测第1次常规  119012
466677
466677
############最终要的是下面的二个人群###############
拆分有保养与没有保养的人群
有保养 ['有首保,1常规,预测第2次常规', '3次及以上常规,预测接下去常规', '有首保,2次常规,预测第3次常规', '有首保,无常规,预测第1次常规', '无首保,1次常规,预测第2次常规(购车)', '无首保,2次常规,预测第3次常规']
没有保养 ['无保养习惯', '无保养记录,预测首保']


In [17]:
print('注意,没有保养记录的人,不能取进店数据!')
print('这个last_baoyang_date是指最后一次保养时间,放在train set中就是true_y,放在test set中就是last_baoyang,然后预测下一次')
def get_vali_data(baoyangdf1):
    baoyangdf1=baoyangdf1.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    print(baoyangdf1['VIN'].nunique())
    
    ydf=baoyangdf1.drop_duplicates(['VIN'],keep='first').reset_index(drop=True)
    y=ydf[['VIN','修理日期','公里数']].drop_duplicates().reset_index(drop=True)
    y.columns=['VIN','last_baoyang_date','last_baoyang_mile']
    print('训练集y的VIN数量:',y['VIN'].nunique())
    
    baoyangdf1=baoyangdf1.merge(y,on=['VIN'],how='left')

    x=baoyangdf1[baoyangdf1['修理日期']<baoyangdf1['last_baoyang_date']] #有保养记录的
    print('训练集VIN数量:',x['VIN'].nunique())
    
    nox=ydf[~(ydf['VIN'].isin(set(x['VIN'])))][['VIN','购车时间','用户属性','一级车系','车龄bin']].drop_duplicates().reset_index(drop=True)
    nox['首保次数']=0
    nox['常规次数']=0
    nox['首保']=0
    nox['常规保养']=0
    print('做成训练集以后,部分只有一次进店保养的人群,就变成了没有保养人群,人数是:')
    print(nox.shape)
    print(nox['VIN'].nunique())
    
    print(baoyangdf1['VIN'].nunique()==x['VIN'].nunique()+nox['VIN'].nunique())
    
    return x,nox,y

注意,没有保养记录的人,不能取进店数据!
这个last_baoyang_date是指最后一次保养时间,放在train set中就是true_y,放在test set中就是last_baoyang,然后预测下一次


In [18]:
print('提取训练集')
x,nox,y=get_vali_data(baoyangdf1)
xx=pd.concat([x,nox],axis=0)
x,nox=fenlei(xx)

提取训练集
415157
训练集y的VIN数量: 415157
训练集VIN数量: 254289
做成训练集以后,部分只有一次进店保养的人群,就变成了没有保养人群,人数是:
(160868, 9)
160868
True
查看是否有没有被分配到的人群
(415157, 3)
415157
VIN      0
人群       0
车龄bin    0
dtype: int64
查看对应的每一类人群的VIN数量
                     人群      人数
0       3次及以上常规,预测接下去常规  120829
1                 无保养习惯  120577
2            无保养记录,预测首保   40291
3  无首保,1次常规,预测第2次常规(购车)    7293
4      无首保,2次常规,预测第3次常规   11873
5       有首保,1常规,预测第2次常规   38524
6      有首保,2次常规,预测第3次常规   28316
7       有首保,无常规,预测第1次常规   47454
415157
415157
############最终要的是下面的二个人群###############
拆分有保养与没有保养的人群
有保养 ['有首保,无常规,预测第1次常规', '3次及以上常规,预测接下去常规', '有首保,1常规,预测第2次常规', '有首保,2次常规,预测第3次常规', '无首保,1次常规,预测第2次常规(购车)', '无首保,2次常规,预测第3次常规']
没有保养 ['无保养习惯', '无保养记录,预测首保']


In [19]:
print(y.columns.tolist())
print(y.head())

['VIN', 'last_baoyang_date', 'last_baoyang_mile']
                 VIN last_baoyang_date  last_baoyang_mile
0  LSVXZ65NXH2102141        2019-09-29             9453.0
1  LSVXZ65NXH2102107        2020-08-03            83112.0
2  LSVXZ65NXH2102009        2018-11-22            14487.0
3  LSVXZ65NXH2101829        2018-12-25            45695.0
4  LSVXZ65NXH2100485        2019-11-14            13807.0


In [20]:
def evaluate(data1,version,rule_name):
    vins=data1['VIN'].nunique()

    if '天数偏差' in list(data1.columns):
        del data1['天数偏差']
    if '公里数偏差' in list(data1.columns):
        del data1['公里数偏差']
    
    data1['天数偏差']=(data1['next_date']-data1['last_baoyang_date']).apply(lambda x:x.days)
    data1['天数偏差']=data1['天数偏差'].apply(abs)
    data1['公里数偏差']=(data1['next_mile']-data1['last_baoyang_mile']).apply(abs)
    
    data1.loc[data1['天数偏差']<=10,'天数偏差人群']='0-10天'
    data1.loc[(data1['天数偏差']>10)&(data1['天数偏差']<=20),'天数偏差人群']='10-20天'
    data1.loc[(data1['天数偏差']>20)&(data1['天数偏差']<=30),'天数偏差人群']='20-30天'
    data1.loc[(data1['天数偏差']>30)&(data1['天数偏差']<=40),'天数偏差人群']='30-40天'
    data1.loc[(data1['天数偏差']>40)&(data1['天数偏差']<=50),'天数偏差人群']='40-50天'
    data1.loc[(data1['天数偏差']>50)&(data1['天数偏差']<=60),'天数偏差人群']='50-60天'
    data1.loc[(data1['天数偏差']>60)&(data1['天数偏差']<=70),'天数偏差人群']='60-70天'
    data1.loc[(data1['天数偏差']>70)&(data1['天数偏差']<=80),'天数偏差人群']='70-80天'
    data1.loc[(data1['天数偏差']>80)&(data1['天数偏差']<=90),'天数偏差人群']='80-90天'
    data1.loc[data1['天数偏差']>90,'天数偏差人群']='90-inf天'

    tmp=data1.groupby('天数偏差人群')['VIN'].nunique().reset_index(name='人数')
    tmp['占比']=tmp['人数']/vins
    tmp['idx']=np.arange(len(tmp))
    print(tmp.head(10))
    
    data1.loc[data1['公里数偏差']<=1000,'公里数偏差人群']='0-1000公里'
    data1.loc[(data1['公里数偏差']>1000)&(data1['公里数偏差']<=2000),'公里数偏差人群']='1000-2000公里'
    data1.loc[(data1['公里数偏差']>2000)&(data1['公里数偏差']<=3000),'公里数偏差人群']='2000-3000公里'
    data1.loc[(data1['公里数偏差']>3000)&(data1['公里数偏差']<=4000),'公里数偏差人群']='3000-4000公里'
    data1.loc[(data1['公里数偏差']>4000)&(data1['公里数偏差']<=5000),'公里数偏差人群']='4000-5000公里'
    data1.loc[(data1['公里数偏差']>5000)&(data1['公里数偏差']<=6000),'公里数偏差人群']='5000-6000公里'
    data1.loc[(data1['公里数偏差']>6000)&(data1['公里数偏差']<=7000),'公里数偏差人群']='6000-7000公里'
    data1.loc[(data1['公里数偏差']>7000)&(data1['公里数偏差']<=8000),'公里数偏差人群']='7000-8000公里'
    data1.loc[(data1['公里数偏差']>8000)&(data1['公里数偏差']<=9000),'公里数偏差人群']='8000-9000公里'
    data1.loc[data1['公里数偏差']>9000,'公里数偏差人群']='9000-公里以上'    
    
    tmpp=data1.groupby('公里数偏差人群')['VIN'].nunique().reset_index(name='人数')
    tmpp['占比']=tmpp['人数']/vins
    tmpp['idx']=np.arange(len(tmpp))
    print(tmpp.head(10))
    
    fin=tmp.merge(tmpp,on='idx',how='left')
    fin['batch']=rule_name
    fin['version']=version
    
    del fin['idx']
    
    return fin

In [21]:
def get_next_day_dayofmile(data3,end_id,is_train):

    if 'daydiff_pred' not in list(data3.columns):
        data3['daydiff_pred']=data3['天数差%d'%end_id]*data3['next_daydiff']
        
    if 'dayofmile_pred' not in list(data3.columns):
        data3['dayofmile_pred']=data3['日均%d'%end_id]*data3['next_dayofmile']
            
    data3.loc[(data3['daydiff_pred']<=0)|(data3['daydiff_pred'].isnull()),'daydiff_pred']=data3[data3['daydiff_pred']>0]['daydiff_pred'].median()
    data3.loc[(data3['dayofmile_pred']<=0)|(data3['dayofmile_pred'].isnull()),'dayofmile_pred']=data3[data3['dayofmile_pred']>0]['dayofmile_pred'].median()
    
    if 'milediff_pred' in list(data3.columns):
        del data3['milediff_pred']
    data3['milediff_pred']=data3['daydiff_pred']*data3['dayofmile_pred']
    
    data3['daydiff_pred']=data3['daydiff_pred'].apply(lambda x:math.ceil(x))
    data3['next_date']=list(map(lambda x,y:x+relativedelta(days=y),data3['修理日期%d'%end_id],data3['daydiff_pred']))
    data3['next_mile']=(data3['公里数%d'%end_id]+data3['milediff_pred']).apply(lambda x:math.ceil(x))
    
    if is_train=='True':
        data3['daydiff_true']=(data3['last_baoyang_date']-data3['修理日期%d'%end_id]).apply(lambda x:x.days)
        data3['milediff_true']=data3['last_baoyang_mile']-data3['公里数%d'%end_id]
        data3['dayofmile_true']=data3['milediff_true']/data3['daydiff_true']
    
    return data3

In [22]:
def get_next_mile_dayofmile(data3,end_id,is_train):

    if 'milediff_pred' not in list(data3.columns):
        data3['milediff_pred']=data3['公里数差%d'%end_id]*data3['next_milediff']
        
    if 'dayofmile_pred' not in list(data3.columns):
        data3['dayofmile_pred']=data3['日均%d'%end_id]*data3['next_dayofmile']
            
    data3.loc[(data3['milediff_pred']<=0)|(data3['milediff_pred'].isnull()),'milediff_pred']=data3[data3['milediff_pred']>0]['milediff_pred'].median()
    data3.loc[(data3['dayofmile_pred']<=0)|(data3['dayofmile_pred'].isnull()),'dayofmile_pred']=data3[data3['dayofmile_pred']>0]['dayofmile_pred'].median()
    
    if 'daydiff_pred' in list(data3.columns):
        del data3['daydiff_pred']
    data3['daydiff_pred']=data3['milediff_pred']/data3['dayofmile_pred']
    
    data3['daydiff_pred']=data3['daydiff_pred'].apply(lambda x:math.ceil(x))
    data3['next_date']=list(map(lambda x,y:x+relativedelta(days=y),data3['修理日期%d'%end_id],data3['daydiff_pred']))
    data3['next_mile']=(data3['公里数%d'%end_id]+data3['milediff_pred']).apply(lambda x:math.ceil(x))
    
    if is_train=='True':
        data3['daydiff_true']=(data3['last_baoyang_date']-data3['修理日期%d'%end_id]).apply(lambda x:x.days)
        data3['milediff_true']=data3['last_baoyang_mile']-data3['公里数%d'%end_id]
        data3['dayofmile_true']=data3['milediff_true']/data3['daydiff_true']
    
    return data3

In [23]:
def get_next_day_mile(data3,end_id,is_train):

    if 'daydiff_pred' not in list(data3.columns):
        data3['daydiff_pred']=data3['天数差%d'%end_id]*data3['next_daydiff']
        
    if 'milediff_pred' not in list(data3.columns):
        data3['milediff_pred']=data3['公里数差%d'%end_id]*data3['next_milediff']
        
    data3.loc[(data3['daydiff_pred']<=0)|(data3['daydiff_pred'].isnull()),'daydiff_pred']=data3[data3['daydiff_pred']>0]['daydiff_pred'].median()
    data3.loc[(data3['milediff_pred']<=0)|(data3['milediff_pred'].isnull()),'milediff_pred']=data3[data3['milediff_pred']>0]['milediff_pred'].median()
    
    data3['daydiff_pred']=data3['daydiff_pred'].apply(lambda x:math.ceil(x))
    data3['next_date']=list(map(lambda x,y:x+relativedelta(days=y),data3['修理日期%d'%end_id],data3['daydiff_pred']))
    data3['next_mile']=(data3['公里数%d'%end_id]+data3['milediff_pred']).apply(lambda x:math.ceil(x))
    
    if 'dayofmile_pred' not in data3.columns.tolist():
        data3['dayofmile_pred']=data3['milediff_pred']/data3['daydiff_pred']
    
    if is_train=='True':
        data3['daydiff_true']=(data3['last_baoyang_date']-data3['修理日期%d'%end_id]).apply(lambda x:x.days)
        data3['milediff_true']=data3['last_baoyang_mile']-data3['公里数%d'%end_id]
        data3['dayofmile_true']=data3['milediff_true']/data3['daydiff_true']
    
    return data3

#### <font color=red>规则人群0:预测首保<br>预测首保得到的daydiff和milediff不适用于迭代,需要放到预测首保到第一次常规中得到这个可迭代的数值;

In [24]:
def get_eva_score(final_vali_1,version,batch_num,rule_name):
    
    mae_day_1=mae(final_vali_1['daydiff_pred'],final_vali_1['daydiff_true'])
    mse_day_1=mse(final_vali_1['daydiff_pred'],final_vali_1['daydiff_true'])
    r_day_1=r2(final_vali_1['daydiff_pred'],final_vali_1['daydiff_true'])
    
    mae_mile_1=mae(final_vali_1['milediff_pred'],final_vali_1['milediff_true'])
    mse_mile_1=mse(final_vali_1['milediff_pred'],final_vali_1['milediff_true'])
    r_mile_1=r2(final_vali_1['milediff_pred'],final_vali_1['milediff_true'])
    
    mae_dayofmile_1=mae(final_vali_1['dayofmile_pred'],final_vali_1['dayofmile_true'])
    mse_dayofmile_1=mse(final_vali_1['dayofmile_pred'],final_vali_1['dayofmile_true'])    
    r_dayofmile_1=r2(final_vali_1['dayofmile_pred'],final_vali_1['dayofmile_true'])
    
    eva_1=pd.DataFrame({'version':[version]})
    eva_1['batch']=batch_num
    eva_1['batch_name']=rule_name
    
    eva_1['mae_day']=mae_day_1
    eva_1['mse_day']=mse_day_1
    eva_1['r2_day']=r_day_1
    
    eva_1['mae_mile']=mae_mile_1
    eva_1['mse_mile']=mse_mile_1
    eva_1['r2_mile']=r_mile_1
    
    eva_1['mae_dayofmile']=mae_dayofmile_1
    eva_1['mse_dayofmile']=mse_dayofmile_1
    eva_1['r2_dayofmile']=r_dayofmile_1
    
    return eva_1

In [25]:
def get_rule_0(nox):
    rule0=nox[nox['人群']=='无保养记录,预测首保']
    rule0['修理日期0']=rule0['购车时间']
    rule0['公里数0']=0
    return rule0

def get_trend_0(x,veh3):
    df0=x[(x['首保次数']==1)]
    df0=df0[df0['首保']==1]
    df0['baoyang_id']=df0.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    df0=get_last_run(df0)
    df0['baoyang_id']=df0['baoyang_id'].astype(int).astype(str)
    df00=pd.pivot(df0,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均']).reset_index()
    df00.columns=[''.join(x) for x in list(df00.columns)]
    df00=df00.merge(veh3,on=['VIN'],how='left')
    return df00

def nihe_0(df00,rule0):

#     lst=['常去经销商']
    lst=['用户属性']
    nihe0=df00.groupby(lst)['天数差1','公里数差1','日均1'].median()
    nihe0.rename(columns={'天数差1':'daydiff_pred','公里数差1':'milediff_pred','日均1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['first_'+x for x in nihe0.columns.tolist()]
    nihe0=nihe0.reset_index()
    rule0=rule0.merge(nihe0,on=lst,how='left')
    
    lst=['一级车系']
    nihe0=df00.groupby(lst)['天数差1','公里数差1','日均1'].median()
    nihe0.rename(columns={'天数差1':'daydiff_pred','公里数差1':'milediff_pred','日均1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['second_'+x for x in nihe0.columns.tolist()]
    nihe0=nihe0.reset_index()
    rule0=rule0.merge(nihe0,on=lst,how='left')
    
#     nihe0=df00.groupby(lst)['天数差1','公里数差1','日均1'].median()
#     nihe0.rename(columns={'天数差1':'daydiff_pred','公里数差1':'milediff_pred','日均1':'dayofmile_pred'},inplace=True)
#     nihe0.columns=['third_'+x for x in nihe0.columns.tolist()]
#     nihe0=nihe0.reset_index()
#     rule0=rule0.merge(nihe0,on=lst,how='left')

    rule0['daydiff_pred']=rule0['first_daydiff_pred']
    rule0.loc[(rule0['daydiff_pred'].isnull())&(rule0['second_daydiff_pred'].notnull()),'daydiff_pred']=rule0[(rule0['daydiff_pred'].isnull())&(rule0['second_daydiff_pred'].notnull())]['second_daydiff_pred']
#     rule0.loc[(rule0['daydiff_pred'].isnull())&(rule0['third_daydiff_pred'].notnull()),'daydiff_pred']=rule0[(rule0['daydiff_pred'].isnull())&(rule0['third_daydiff_pred'].notnull())]['third_daydiff_pred']

    rule0['milediff_pred']=rule0['first_milediff_pred']
    rule0.loc[(rule0['milediff_pred'].isnull())&(rule0['second_milediff_pred'].notnull()),'milediff_pred']=rule0[(rule0['milediff_pred'].isnull())&(rule0['second_milediff_pred'].notnull())]['second_milediff_pred']
#     rule0.loc[(rule0['milediff_pred'].isnull())&(rule0['third_milediff_pred'].notnull()),'milediff_pred']=rule0[(rule0['milediff_pred'].isnull())&(rule0['third_milediff_pred'].notnull())]['third_milediff_pred']

    rule0['dayofmile_pred']=rule0['first_dayofmile_pred']
    rule0.loc[(rule0['dayofmile_pred'].isnull())&(rule0['second_dayofmile_pred'].notnull()),'dayofmile_pred']=rule0[(rule0['dayofmile_pred'].isnull())&(rule0['second_dayofmile_pred'].notnull())]['second_dayofmile_pred']
#     rule0.loc[(rule0['dayofmile_pred'].isnull())&(rule0['third_dayofmile_pred'].notnull()),'dayofmile_pred']=rule0[(rule0['dayofmile_pred'].isnull())&(rule0['third_dayofmile_pred'].notnull())]['third_dayofmile_pred']

    print('检查缺失情况')
    print(pd.isnull(rule0[['VIN','daydiff_pred','milediff_pred','dayofmile_pred']]).sum())
    
    print('检查脏数据情况:')
    print('预测出来的天数差<=0',len(rule0[rule0['daydiff_pred']<=0]))
    print('预测出来的公里数差<=0',len(rule0[rule0['milediff_pred']<=0]))
    print('预测出来的日均<=0',len(rule0[rule0['dayofmile_pred']<=0]))
    
    rule0.loc[rule0['daydiff_pred']<=0,'daydiff_pred']=rule0[rule0['daydiff_pred']>0]['daydiff_pred'].median()
    rule0.loc[rule0['milediff_pred']<=0,'milediff_pred']=rule0[rule0['milediff_pred']>0]['milediff_pred'].median()
    rule0.loc[rule0['dayofmile_pred']<=0,'dayofmile_pred']=rule0[rule0['dayofmile_pred']>0]['dayofmile_pred'].median()

    return rule0


def get_final_rule_0(baoyangdf1,nobaoyang1,x,nox,y,veh3):
    rule_name='无保养记录,预测首保'
    batch=0

    print("######提取预测集#######")
    df11=get_trend_0(baoyangdf1,veh3)
    rule11=get_rule_0(nobaoyang1)
    rule111=nihe_0(df11,rule11)
    
    print('##########提取训练集#######')
    valix=get_trend_0(x,veh3)
    valiy=get_rule_0(nox)
    validf=nihe_0(valix,valiy)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    end_id=0
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [26]:
_=get_final_rule_0(baoyangdf1,nobaoyang1,x,nox,y,veh2) #预测首保

######提取预测集#######
检查缺失情况
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
检查脏数据情况:
预测出来的天数差<=0 0
预测出来的公里数差<=0 0
预测出来的日均<=0 0
##########提取训练集#######
检查缺失情况
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
检查脏数据情况:
预测出来的天数差<=0 0
预测出来的公里数差<=0 0
预测出来的日均<=0 0
##########train结果#########
    天数偏差人群     人数        占比  idx
0    0-10天   4231  0.105011    0
1   10-20天   4023  0.099849    1
2   20-30天   3914  0.097143    2
3   30-40天   3576  0.088754    3
4   40-50天   3265  0.081035    4
5   50-60天   3084  0.076543    5
6   60-70天   2870  0.071232    6
7   70-80天   2387  0.059244    7
8   80-90天   2004  0.049738    8
9  90-inf天  10937  0.271450    9
       公里数偏差人群     人数        占比  idx
0     0-1000公里  23925  0.593805    0
1  1000-2000公里   9215  0.228711    1
2  2000-3000公里   3774  0.093669    2
3  3000-4000公里   1312  0.032563    3
4  4000-5000公里    508  0.012608    4
5  5000-6000公里    408  0.010126    5
6  6000

#### <font color=red>规则人群1:没有保养习惯的人

In [27]:
def get_trend_1(baoyangdf1,veh3):
    print('#取出车龄>3的人群的,在车龄>3之后的第一次进店数据,根据车系喂入')
    dff1=baoyangdf1[baoyangdf1['车龄bin']>3]
    dff1['进店车龄']=(dff1['修理日期']-dff1['购车时间']).apply(lambda x:x.days)
    dff1=dff1[dff1['进店车龄']>180*3]
    dff1=dff1.sort_values(by=['VIN','修理日期'],ascending=False).drop_duplicates(['VIN'],keep='last').reset_index()
    if 'baoyang_id' in list(dff1.columns):
        del dff1['baoyang_id']
    dff1['baoyang_id']=dff1.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    dff1=get_last_run(dff1)

    dff1['baoyang_id']=dff1['baoyang_id'].astype(int).astype(str)
    dff11=pd.pivot(dff1,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均']).reset_index()
    dff11.columns=[''.join(x) for x in list(dff11.columns)]
    dff11=dff11.merge(veh3,on=['VIN'],how='left')
    
    return dff11

def get_rule_1(nobaoyang1):
    rule1=nobaoyang1[nobaoyang1['人群']=='无保养习惯']
    rule1['修理日期0']=rule1['购车时间']
    rule1['公里数0']=0
    print(rule1.shape)
    print(rule1['VIN'].nunique())
    return rule1

def nihe_1(df00,rule0):

    lst=['一级车系']
    nihe0=df00.groupby(lst)['天数差1','公里数差1','日均1'].median()
    nihe0.rename(columns={'天数差1':'daydiff_pred','公里数差1':'milediff_pred','日均1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['first_'+x for x in nihe0.columns.tolist()]
    nihe0=nihe0.reset_index()
    rule0=rule0.merge(nihe0,on=lst,how='left')

    rule0['daydiff_pred']=rule0['first_daydiff_pred']
    rule0['milediff_pred']=rule0['first_milediff_pred']
    rule0['dayofmile_pred']=rule0['first_dayofmile_pred']

    print('检查缺失情况')
    print(pd.isnull(rule0[['VIN','daydiff_pred','milediff_pred','dayofmile_pred']]).sum())
    
    print('检查脏数据情况:')
    print('预测出来的天数差<=0',len(rule0[rule0['daydiff_pred']<=0]))
    print('预测出来的公里数差<=0',len(rule0[rule0['milediff_pred']<=0]))
    print('预测出来的日均<=0',len(rule0[rule0['dayofmile_pred']<=0]))
    
    rule0.loc[rule0['daydiff_pred']<=0,'daydiff_pred']=rule0[rule0['daydiff_pred']>0]['daydiff_pred'].median()
    rule0.loc[rule0['milediff_pred']<=0,'milediff_pred']=rule0[rule0['milediff_pred']>0]['milediff_pred'].median()
    rule0.loc[rule0['dayofmile_pred']<=0,'dayofmile_pred']=rule0[rule0['dayofmile_pred']>0]['dayofmile_pred'].median()

    return rule0

def get_final_rule_1(baoyangdf1,nobaoyang1,veh3):
    rule_name='无保养习惯'
    batch=1
    
    print('没有保养习惯的人群,没有y,因此直接拟合即可')
    print("######提取预测集#######")
    df11=get_trend_1(baoyangdf1,veh3)
    rule11=get_rule_1(nobaoyang1)
    rule111=nihe_1(df11,rule11)
    
    print('##########test结果#########')
    end_id=0
    
    rule111.loc[rule111['milediff_pred'].isnull(),'milediff_pred']=rule111['milediff_pred'].median()
    rule111.loc[rule111['dayofmile_pred'].isnull(),'dayofmile_pred']=rule111['dayofmile_pred'].median()
    
    is_train='False'
    final_rule=get_next_day_mile(rule111,end_id,is_train)
    final_rule['最后一次保养日期']=final_rule['修理日期%d'%end_id]
    final_rule['最后一次保养公里数']=final_rule['公里数%d'%end_id]
    final_rule['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule=final_rule[cols].drop_duplicates()    
    final_rule.to_pickle(savep+'%d_rule_test.pkl'%batch)
    
    return []

In [28]:
_=get_final_rule_1(baoyangdf1,nobaoyang1,veh2)

没有保养习惯的人群,没有y,因此直接拟合即可
######提取预测集#######
#取出车龄>3的人群的,在车龄>3之后的第一次进店数据,根据车系喂入
(71823, 65)
32268
检查缺失情况
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
检查脏数据情况:
预测出来的天数差<=0 0
预测出来的公里数差<=0 0
预测出来的日均<=0 0
##########test结果#########


#### <font color=red>规则人群2:有首保,无常规,预测第1次常规

In [29]:
def get_trend_2(baoyangdf,veh2):
    baoyangdf=baoyangdf.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df1=baoyangdf[(baoyangdf['首保次数']==1)&(baoyangdf['常规次数']>=1)]
    if 'baoyang_id' in list(df1.columns):
        del df1['baoyang_id']
    df1['baoyang_id']=df1.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    df1=df1[df1['baoyang_id']<=2]
    df1=get_last_run(df1)
    
    df1['baoyang_id']=df1['baoyang_id'].astype(int).astype(str)
    df11=pd.pivot(df1,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均','经销商代码']).reset_index()
    df11.columns=[''.join(x) for x in df11.columns.tolist()]

    df11['daydiff_2_1']=df11['天数差2']/df11['天数差1']
    df11['milediff_2_1']=df11['公里数差2']/(df11['公里数差1']+1)
    df11['dayofmile_2_1']=df11['日均2']/(df11['日均1']+0.01)
    
    df11['天数差1bin']=df11['天数差1'].apply(lambda x:math.ceil(x/15))
    df11['天数差2bin']=df11['天数差2'].apply(lambda x:math.ceil(x/15))
    df11['公里数差1bin']=df11['公里数差1'].apply(lambda x:math.ceil(x/2500))
    df11['公里数差2bin']=df11['公里数差2'].apply(lambda x:math.ceil(x/2500))

    df11=df11.merge(veh2,on=['VIN'],how='left')
    
    return df11

def get_rule_2(baoyangdf,veh2):
    rule1=baoyangdf[baoyangdf['人群']=='有首保,无常规,预测第1次常规']
    print(rule1.shape)
    print(rule1['VIN'].nunique())
    
    rule1=get_last_run(rule1)
    if 'baoyang_id' in list(rule1.columns):
        del rule1['baoyang_id']
    rule1['baoyang_id']=rule1.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    print(rule1['baoyang_id'].max())
    
    rule1['baoyang_id']=rule1['baoyang_id'].astype(int).astype(str)
    rule11=pd.pivot(rule1,index='VIN',columns='baoyang_id',values=['修理日期','天数差','公里数','公里数差','日均','经销商代码']).reset_index()
    rule11.columns=[''.join(x) for x in rule11.columns.tolist()]
    rule11['天数差1bin']=rule11['天数差1'].apply(lambda x:math.ceil(x/15))
    rule11['公里数差1bin']=rule11['公里数差1'].apply(lambda x:math.ceil(x/2500))

    rule11=rule11.merge(veh2,on=['VIN'],how='left')
    return rule11

def nihe_2(df11,rule11):

    for col in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']:
        df11[col]=df11[col].astype(float)
        
    lst=['天数差1bin','公里数差1bin']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['first_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    lst=['天数差1bin','使用性质']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['second_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    lst=['天数差1bin']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['third_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    rule11['next_daydiff']=rule11['first_daydiff_2_1']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_2_1'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_2_1'].notnull())]['second_daydiff_2_1']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_2_1'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_2_1'].notnull())]['third_daydiff_2_1']

    rule11['next_milediff']=rule11['first_milediff_2_1']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['second_milediff_2_1'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['second_milediff_2_1'].notnull())]['second_milediff_2_1']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['third_milediff_2_1'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['third_milediff_2_1'].notnull())]['third_milediff_2_1']

    rule11['next_dayofmile']=rule11['first_dayofmile_2_1']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_2_1'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_2_1'].notnull())]['second_dayofmile_2_1']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_2_1'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_2_1'].notnull())]['third_dayofmile_2_1']
    
    daydiff_median=df11['daydiff_2_1'].median()
#     print(daydiff_median)
    milediff_median=df11['milediff_2_1'].median()
#     print(milediff_median)
    dayofmile_median=df11['dayofmile_2_1'].median()
#     print(dayofmile_median)
    
    rule11.loc[rule11['next_daydiff'].isnull(),'next_daydiff']=daydiff_median
    rule11.loc[rule11['next_milediff'].isnull(),'next_milediff']=milediff_median
    rule11.loc[rule11['next_dayofmile'].isnull(),'next_dayofmile']=dayofmile_median

    print("中位数填充以后的缺失情况:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())
    
#     print("左右平滑")
#     for col in ['next_daydiff','next_milediff','next_dayofmile']:
#         print(col)
#         print(rule11[col].quantile(0.01))
#         print(rule11[col].quantile(0.99))
#         rule11.loc[rule11[col]>=rule11[col].quantile(0.02),col]=rule11[col].quantile(0.02)
#         rule11.loc[rule11[col]<=rule11[col].quantile(0.98),col]=rule11[col].quantile(0.98)
#         print('-'*30)
    return rule11

def get_final_rule_2(baoyangdf1,x,y,veh2):
    rule_name='有首保,无常规,预测第1次常规'
    batch=2
    
    end_id=1
    
    print("######提取预测集#######")
    df11=get_trend_2(baoyangdf1,veh2)
    rule11=get_rule_2(baoyangdf1,veh2)
    rule111=nihe_2(df11,rule11)
    
    print('##########提取训练集#######')
    valix=get_trend_2(x,veh2)
    valiy=get_rule_2(x,veh2)
    validf=nihe_2(valix,valiy)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [30]:
_=get_final_rule_2(baoyangdf1,x,y,veh2)

######提取预测集#######
(119012, 63)
119012
1.0
中位数填充以后的缺失情况:
119012
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########提取训练集#######
(47454, 65)
47454
1.0
中位数填充以后的缺失情况:
47454
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########train结果#########
    天数偏差人群     人数        占比  idx
0    0-10天   4857  0.102352    0
1   10-20天   4547  0.095819    1
2   20-30天   4490  0.094618    2
3   30-40天   4056  0.085472    3
4   40-50天   3688  0.077717    4
5   50-60天   3235  0.068171    5
6   60-70天   2793  0.058857    6
7   70-80天   2339  0.049290    7
8   80-90天   2022  0.042610    8
9  90-inf天  15427  0.325094    9
       公里数偏差人群     人数        占比  idx
0     0-1000公里  16359  0.344734    0
1  1000-2000公里  13429  0.282990    1
2  2000-3000公里   7541  0.158912    2
3  3000-4000公里   3844  0.081005    3
4  4000-5000公里   2250  0.047414    4
5  5000-6000公里   1275  0.026868    5
6  6000-7000公里    756  0.015931    6

#### <font color=red>规则人群3:无首保,1次常规,预测第2次常规(购车)

In [31]:
def get_trend_3(baoyangdf,veh2):
    df22=baoyangdf[baoyangdf['常规次数']>=2]
    df22=df22[df22['首保']!=1]
    df22['baoyang_id']=df22.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    df22=df22[df22['baoyang_id']<=2]
    
    df22=get_last_run(df22)
    df22['baoyang_id']=df22['baoyang_id'].astype(int).astype(str)
    df11=pd.pivot(df22,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均','经销商代码']).reset_index()
    df11.columns=[''.join(x) for x in df11.columns.tolist()]
    
    df11['daydiff_2_1']=df11['天数差2']/(df11['天数差1'])
    df11['milediff_2_1']=df11['公里数差2']/(df11['公里数差1']+1)
    df11['dayofmile_2_1']=df11['日均2']/(df11['日均1']+0.01)
    
    df11['天数差1bin']=df11['天数差1'].apply(lambda x:math.ceil(x/15))
    df11['天数差2bin']=df11['天数差2'].apply(lambda x:math.ceil(x/15))
    df11['公里数差1bin']=df11['公里数差1'].apply(lambda x:math.ceil(x/2500))
    df11['公里数差2bin']=df11['公里数差2'].apply(lambda x:math.ceil(x/2500))
    df11=df11.merge(veh2,on=['VIN'],how='left')

    return df11

def get_rule_3(baoyangdf,veh2):
    rule2=baoyangdf[baoyangdf['人群']=='无首保,1次常规,预测第2次常规(购车)']
    rule2=get_last_run(rule2)
    rule2['baoyang_id']=rule2.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    rule2['baoyang_id']=rule2['baoyang_id'].astype(int).astype(str)
    rule22=pd.pivot(rule2,index='VIN',columns='baoyang_id',values=['修理日期','天数差','公里数','公里数差','日均','经销商代码']).reset_index()
    rule22.columns=[''.join(x) for x in rule22.columns.tolist()]
    rule22['天数差1bin']=rule22['天数差1'].apply(lambda x:math.ceil(x/15))
    rule22['公里数差1bin']=rule22['公里数差1'].apply(lambda x:math.ceil(x/2500))
    rule22=rule22.merge(veh2,on=['VIN'],how='left')
    return rule22

def nihe_3(df11,rule11):

    for col in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']:
        df11[col]=df11[col].astype(float)
        
    lst=['天数差1bin','公里数差1bin']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['first_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    lst=['天数差1bin','使用性质']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['second_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    lst=['天数差1bin']
    df111=df11.groupby(lst)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    df111.columns=['third_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
    
    rule11['next_daydiff']=rule11['first_daydiff_2_1']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_2_1'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_2_1'].notnull())]['second_daydiff_2_1']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_2_1'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_2_1'].notnull())]['third_daydiff_2_1']

    rule11['next_milediff']=rule11['first_milediff_2_1']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['second_milediff_2_1'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['second_milediff_2_1'].notnull())]['second_milediff_2_1']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['third_milediff_2_1'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['third_milediff_2_1'].notnull())]['third_milediff_2_1']

    rule11['next_dayofmile']=rule11['first_dayofmile_2_1']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_2_1'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_2_1'].notnull())]['second_dayofmile_2_1']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_2_1'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_2_1'].notnull())]['third_dayofmile_2_1']
    
    daydiff_median=df11['daydiff_2_1'].median()
#     print(daydiff_median)
    milediff_median=df11['milediff_2_1'].median()
#     print(milediff_median)
    dayofmile_median=df11['dayofmile_2_1'].median()
#     print(dayofmile_median)
    
#     print("检查没有拟合到的数量:")
#     print(rule11['VIN'].nunique())
#     print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())
    
    rule11.loc[rule11['next_daydiff'].isnull(),'next_daydiff']=daydiff_median
    rule11.loc[rule11['next_milediff'].isnull(),'next_milediff']=milediff_median
    rule11.loc[rule11['next_dayofmile'].isnull(),'next_dayofmile']=dayofmile_median

    print("中位数填充以后的缺失情况:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())

#     print("左右平滑")
#     for col in ['next_daydiff','next_milediff','next_dayofmile']:
#         print(col)
#         print(rule11[col].quantile(0.01))
#         print(rule11[col].quantile(0.99))
#         rule11.loc[rule11[col]>=rule11[col].quantile(0.02),col]=rule11[col].quantile(0.02)
#         rule11.loc[rule11[col]<=rule11[col].quantile(0.98),col]=rule11[col].quantile(0.98)
#         print('-'*30)
    return rule11

def get_final_rule_3(baoyangdf1,x,y,veh2):
    rule_name='无首保,1次常规,预测第2次常规(购车)'
    batch=3
    
    end_id=1
    
    print("######提取预测集#######")
    df11=get_trend_3(baoyangdf1,veh2)
    rule11=get_rule_3(baoyangdf1,veh2)
    rule111=nihe_3(df11,rule11)
    
    print('##########提取训练集#######')
    valix=get_trend_3(x,veh2)
    valiy=get_rule_3(x,veh2)
    validf=nihe_3(valix,valiy)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [32]:
_=get_final_rule_3(baoyangdf1,x,y,veh2)

######提取预测集#######
中位数填充以后的缺失情况:
41856
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########提取训练集#######
中位数填充以后的缺失情况:
7293
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########train结果#########
    天数偏差人群    人数        占比  idx
0    0-10天   395  0.054162    0
1   10-20天   331  0.045386    1
2   20-30天   326  0.044700    2
3   30-40天   306  0.041958    3
4   40-50天   295  0.040450    4
5   50-60天   275  0.037707    5
6   60-70天   231  0.031674    6
7   70-80天   210  0.028795    7
8   80-90天   174  0.023858    8
9  90-inf天  4750  0.651309    9
       公里数偏差人群    人数        占比  idx
0     0-1000公里  1198  0.164267    0
1  1000-2000公里  1082  0.148361    1
2  2000-3000公里   790  0.108323    2
3  3000-4000公里   577  0.079117    3
4  4000-5000公里   380  0.052105    4
5  5000-6000公里   276  0.037845    5
6  6000-7000公里   232  0.031811    6
7  7000-8000公里   197  0.027012    7
8  8000-9000公里   148  0.020293

#### <font color=red>规则人群4:有首保,1常规,预测第2次常规

In [33]:
def get_trend_4(baoyangdf,veh2):
    df22=baoyangdf[(baoyangdf['首保次数']==1)&(baoyangdf['常规次数']>=2)]
    df22['baoyang_id']=df22.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    df22=df22[df22['baoyang_id']<=3]
    
    df22=get_last_run(df22)
    df22['baoyang_id']=df22['baoyang_id'].astype(int).astype(str)
    
    df11=pd.pivot(df22,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均','经销商代码']).reset_index()
    df11.columns=[''.join(x) for x in df11.columns.tolist()]

    df11['daydiff_2_1']=df11['天数差2']/(df11['天数差1'])
    df11['milediff_2_1']=df11['公里数差2']/(df11['公里数差1']+1)
    df11['dayofmile_2_1']=df11['日均2']/(df11['日均1']+0.01)
    
    df11['daydiff_2_1_bin']=df11['daydiff_2_1'].apply(lambda x:round(x,1))
    df11['milediff_2_1_bin']=df11['milediff_2_1'].apply(lambda x:round(x,1))
    df11['dayofmile_2_1_bin']=df11['dayofmile_2_1'].apply(lambda x:round(x,1))
    
    df11['daydiff_3_2']=df11['天数差3']/df11['天数差2']
    df11['milediff_3_2']=df11['公里数差3']/(df11['公里数差2']+1)
    df11['dayofmile_3_2']=df11['日均3']/(df11['日均2']+0.01)   
    
    df11['天数差1bin']=df11['天数差1'].apply(lambda x:math.ceil(x/15))
    df11['天数差2bin']=df11['天数差2'].apply(lambda x:math.ceil(x/15))
    df11['天数差3bin']=df11['天数差3'].apply(lambda x:math.ceil(x/15))
    
    df11['公里数差1bin']=df11['公里数差1'].apply(lambda x:math.ceil(x/2500))
    df11['公里数差2bin']=df11['公里数差2'].apply(lambda x:math.ceil(x/2500))
    df11['公里数差3bin']=df11['公里数差3'].apply(lambda x:math.ceil(x/2500))
    
    df11=df11.merge(veh2,on=['VIN'],how='left')
    return df11

def get_rule_4(baoyangdf,veh2):
    rule3=baoyangdf[baoyangdf['人群']=='有首保,1常规,预测第2次常规']
    rule3['baoyang_id']=rule3.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    rule3=get_last_run(rule3)
    
    rule3['baoyang_id']=rule3['baoyang_id'].astype(int).astype(str)

    rule33=pd.pivot(rule3,index='VIN',columns='baoyang_id',values=['修理日期','公里数','天数差','公里数差','日均','经销商代码'])
    rule33=rule33.reset_index()
    rule33.columns=[''.join(x) for x in rule33.columns]
    
    rule33['天数差1bin']=rule33['天数差1'].apply(lambda x:math.ceil(x/15))
    rule33['天数差2bin']=rule33['天数差2'].apply(lambda x:math.ceil(x/15))
    rule33['公里数差1bin']=rule33['公里数差1'].apply(lambda x:math.ceil(x/2500))
    rule33['公里数差2bin']=rule33['公里数差2'].apply(lambda x:math.ceil(x/2500))
    
    rule33['daydiff_2_1']=rule33['天数差2']/(rule33['天数差1'])
    rule33['milediff_2_1']=rule33['公里数差2']/(rule33['公里数差1']+1)
    rule33['dayofmile_2_1']=rule33['日均2']/(rule33['日均1']+0.01)
    
    rule33['daydiff_2_1_bin']=rule33['daydiff_2_1'].apply(lambda x:round(x,1))
    rule33['milediff_2_1_bin']=rule33['milediff_2_1'].apply(lambda x:round(x,1))
    rule33['dayofmile_2_1_bin']=rule33['dayofmile_2_1'].apply(lambda x:round(x,1))
    
    rule33=rule33.merge(veh2,on=['VIN'],how='left')
    return rule33

def nihe_4(df11,rule11):
    for col in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']:
        df11[col]=df11[col].astype(float)
        
    lst=['天数差1bin','公里数差1bin','天数差2bin','公里数差2bin']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['first_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','first_daydiff_3_2','first_milediff_3_2','first_dayofmile_3_2']]).sum())
    
    lst=['天数差1bin','天数差2bin','一级车系']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['second_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','second_daydiff_3_2','second_milediff_3_2','second_dayofmile_3_2']]).sum())
    
    lst=['天数差1bin','天数差2bin']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['third_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','third_daydiff_3_2','third_milediff_3_2','third_dayofmile_3_2']]).sum())
    
    rule11['next_daydiff']=rule11['first_daydiff_3_2']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_3_2'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_3_2'].notnull())]['second_daydiff_3_2']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_3_2'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_3_2'].notnull())]['third_daydiff_3_2']

    rule11['next_milediff']=rule11['first_milediff_3_2']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['second_milediff_3_2'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['second_milediff_3_2'].notnull())]['second_milediff_3_2']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['third_milediff_3_2'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['third_milediff_3_2'].notnull())]['third_milediff_3_2']

    rule11['next_dayofmile']=rule11['first_dayofmile_3_2']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_3_2'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_3_2'].notnull())]['second_dayofmile_3_2']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_3_2'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_3_2'].notnull())]['third_dayofmile_3_2']
    
    daydiff_median=df11['daydiff_3_2'].median()
#     print(daydiff_median)
    milediff_median=df11['milediff_3_2'].median()
#     print(milediff_median)
    dayofmile_median=df11['dayofmile_3_2'].median()
#     print(dayofmile_median)
    
    print("检查没有拟合到的数量:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())
    
    rule11.loc[rule11['next_daydiff'].isnull(),'next_daydiff']=daydiff_median
    rule11.loc[rule11['next_milediff'].isnull(),'next_milediff']=milediff_median
    rule11.loc[rule11['next_dayofmile'].isnull(),'next_dayofmile']=dayofmile_median

    print("中位数填充以后的缺失情况:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())
    
#     for col in ['next_daydiff','next_milediff','next_dayofmile']:
#         rule11.loc[rule11[col]>=rule11[col].quantile(0.02),col]=rule11[col].quantile(0.02)
#         rule11.loc[rule11[col]<=rule11[col].quantile(0.98),col]=rule11[col].quantile(0.98)
    return rule11

def get_final_rule_4(baoyangdf1,x,y,veh2):
    rule_name='有首保,1常规,预测第2次常规'
    batch=4
    
    end_id=2
    
    print("######提取预测集#######")
    df11=get_trend_4(baoyangdf1,veh2)
    rule11=get_rule_4(baoyangdf1,veh2)
    rule111=nihe_4(df11,rule11)
    
    print('##########搜索最佳参数#######')
    valix=get_trend_4(x,veh2)
    valiy=get_rule_4(x,veh2)
    validf=nihe_4(valix,valiy)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [34]:
_=get_final_rule_4(baoyangdf1,x,y,veh2)

######提取预测集#######
检查没有拟合到的数量:
47602
VIN                 0
next_daydiff      392
next_milediff     392
next_dayofmile    392
dtype: int64
中位数填充以后的缺失情况:
47602
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########搜索最佳参数#######
检查没有拟合到的数量:
38524
VIN                 0
next_daydiff      564
next_milediff     564
next_dayofmile    564
dtype: int64
中位数填充以后的缺失情况:
38524
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########train结果#########
    天数偏差人群     人数        占比  idx
0    0-10天   3177  0.082468    0
1   10-20天   2810  0.072942    1
2   20-30天   2746  0.071280    2
3   30-40天   2589  0.067205    3
4   40-50天   2309  0.059937    4
5   50-60天   2078  0.053940    5
6   60-70天   1873  0.048619    6
7   70-80天   1733  0.044985    7
8   80-90天   1436  0.037275    8
9  90-inf天  17773  0.461349    9
       公里数偏差人群    人数        占比  idx
0     0-1000公里  9016  0.234036    0
1  1000-2000公里  7848  0.203717 

#### <font color=red>规则人群5:无首保,2次常规,预测第3次常规

In [35]:
def get_trend_5(baoyangdf,veh2):
    df22=baoyangdf[(baoyangdf['常规次数']>=3)]
    df22=df22[df22['首保']!=1]
    df22['baoyang_id']=df22.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    df22=df22[df22['baoyang_id']<=3]
#     print(df22.shape)
#     print(df22['VIN'].nunique())
    
    df22=get_last_run(df22)
    
    df22['baoyang_id']=df22['baoyang_id'].astype(int).astype(str)
    df11=pd.pivot(df22,index='VIN',columns='baoyang_id',values=['天数差','公里数差','日均','经销商代码'])
    df11=df11.reset_index()
    df11.columns=[''.join(x) for x in df11.columns.tolist()]

    df11['daydiff_2_1']=df11['天数差2']/(df11['天数差1'])
    df11['milediff_2_1']=df11['公里数差2']/(df11['公里数差1']+1)
    df11['dayofmile_2_1']=df11['日均2']/(df11['日均1']+0.01)
    
    df11['daydiff_3_2']=df11['天数差3']/df11['天数差2']
    df11['milediff_3_2']=df11['公里数差3']/(df11['公里数差2']+1)
    df11['dayofmile_3_2']=df11['日均3']/(df11['日均2']+0.01)   
    
    df11['天数差1bin']=df11['天数差1'].apply(lambda x:math.ceil(x/15))
    df11['天数差2bin']=df11['天数差2'].apply(lambda x:math.ceil(x/15))
    df11['天数差3bin']=df11['天数差3'].apply(lambda x:math.ceil(x/15))
    df11['公里数差1bin']=df11['公里数差1'].apply(lambda x:math.ceil(x/2500))
    df11['公里数差2bin']=df11['公里数差2'].apply(lambda x:math.ceil(x/2500))

    df11=df11.merge(veh2,on=['VIN'],how='left')

    return df11

def get_rule_5(baoyangdf,veh2):
    rule4=baoyangdf[baoyangdf['人群']=='无首保,2次常规,预测第3次常规']
#     print(rule4.shape)
#     print(rule4['VIN'].nunique())
    rule4['baoyang_id']=rule4.groupby('VIN')['修理日期'].rank(method='dense',ascending=True)
    rule4=get_last_run(rule4)
    
    rule4['baoyang_id']=rule4['baoyang_id'].astype(int).astype(str)
    rule44=pd.pivot(rule4,index='VIN',columns='baoyang_id',values=['修理日期','公里数','天数差','公里数差','日均','经销商代码'])
    rule44=rule44.reset_index()
    rule44.columns=[''.join(x) for x in rule44.columns.tolist()]
    
    rule44['天数差1bin']=rule44['天数差1'].apply(lambda x:math.ceil(x/15))
    rule44['天数差2bin']=rule44['天数差2'].apply(lambda x:math.ceil(x/15))
    rule44['公里数差1bin']=rule44['公里数差1'].apply(lambda x:math.ceil(x/2500))
    rule44['公里数差2bin']=rule44['公里数差2'].apply(lambda x:math.ceil(x/2500))
    
    rule44=rule44.merge(veh2,on=['VIN'],how='left')
    return rule44

def nihe_5(df11,rule11):

    for col in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']:
        df11[col]=df11[col].astype(float)
        
    lst=['天数差1bin','公里数差1bin','天数差2bin','公里数差2bin']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['first_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','first_daydiff_3_2','first_milediff_3_2','first_dayofmile_3_2']]).sum())
    
    lst=['天数差1bin','天数差2bin','一级车系']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['second_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','second_daydiff_3_2','second_milediff_3_2','second_dayofmile_3_2']]).sum())
    
    lst=['天数差1bin','天数差2bin']
    df111=df11.groupby(lst)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    df111.columns=['third_'+x for x in df111.columns.tolist()]
    df111=df111.reset_index()
    rule11=rule11.merge(df111,on=lst,how='left')
#     print("检查拟合数量:")
#     print(pd.isnull(rule11[['VIN','third_daydiff_3_2','third_milediff_3_2','third_dayofmile_3_2']]).sum())
    
    rule11['next_daydiff']=rule11['first_daydiff_3_2']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_3_2'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['second_daydiff_3_2'].notnull())]['second_daydiff_3_2']
    rule11.loc[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_3_2'].notnull()),'next_daydiff']=rule11[(rule11['next_daydiff'].isnull())&(rule11['third_daydiff_3_2'].notnull())]['third_daydiff_3_2']

    rule11['next_milediff']=rule11['first_milediff_3_2']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['second_milediff_3_2'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['second_milediff_3_2'].notnull())]['second_milediff_3_2']
    rule11.loc[(rule11['next_milediff'].isnull())&(rule11['third_milediff_3_2'].notnull()),'next_milediff']=rule11[(rule11['next_milediff'].isnull())&(rule11['third_milediff_3_2'].notnull())]['third_milediff_3_2']

    rule11['next_dayofmile']=rule11['first_dayofmile_3_2']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_3_2'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['second_dayofmile_3_2'].notnull())]['second_dayofmile_3_2']
    rule11.loc[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_3_2'].notnull()),'next_dayofmile']=rule11[(rule11['next_dayofmile'].isnull())&(rule11['third_dayofmile_3_2'].notnull())]['third_dayofmile_3_2']
    
    daydiff_median=df11['daydiff_3_2'].median()
#     print(daydiff_median)
    milediff_median=df11['milediff_3_2'].median()
#     print(milediff_median)
    dayofmile_median=df11['dayofmile_3_2'].median()
#     print(dayofmile_median)
    
    print("检查没有拟合到的数量:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())
    
    rule11.loc[rule11['next_daydiff'].isnull(),'next_daydiff']=daydiff_median
    rule11.loc[rule11['next_milediff'].isnull(),'next_milediff']=milediff_median
    rule11.loc[rule11['next_dayofmile'].isnull(),'next_dayofmile']=dayofmile_median

    print("中位数填充以后的缺失情况:")
    print(rule11['VIN'].nunique())
    print(pd.isnull(rule11[['VIN','next_daydiff','next_milediff','next_dayofmile']]).sum())

    return rule11

def get_final_rule_5(baoyangdf1,x,y,veh2):
    rule_name='无首保,2次常规,预测第3次常规'
    batch=5
    
    end_id=2
    
    print("######提取预测集#######")
    df11=get_trend_5(baoyangdf1,veh2)
    rule11=get_rule_5(baoyangdf1,veh2)
    rule111=nihe_5(df11,rule11)
    
    print('##########搜索最佳参数#######')
    valix=get_trend_5(x,veh2)
    valiy=get_rule_5(x,veh2)
    validf=nihe_5(valix,valiy)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [36]:
_=get_final_rule_5(baoyangdf1,x,y,veh2)

######提取预测集#######
检查没有拟合到的数量:
7145
VIN                  0
next_daydiff      1570
next_milediff     1570
next_dayofmile    1570
dtype: int64
中位数填充以后的缺失情况:
7145
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########搜索最佳参数#######
检查没有拟合到的数量:
11873
VIN                  0
next_daydiff      1569
next_milediff     1569
next_dayofmile    1569
dtype: int64
中位数填充以后的缺失情况:
11873
VIN               0
next_daydiff      0
next_milediff     0
next_dayofmile    0
dtype: int64
##########train结果#########
    天数偏差人群    人数        占比  idx
0    0-10天   731  0.061568    0
1   10-20天   681  0.057357    1
2   20-30天   676  0.056936    2
3   30-40天   604  0.050872    3
4   40-50天   577  0.048598    4
5   50-60天   517  0.043544    5
6   60-70天   448  0.037733    6
7   70-80天   420  0.035374    7
8   80-90天   350  0.029479    8
9  90-inf天  6869  0.578540    9
       公里数偏差人群    人数        占比  idx
0     0-1000公里  2102  0.177040    0
1  1000-2000公里  1650  0.138971    1


#### <font color=red>规则人群6:有首保,2次常规,预测第3次常规

In [37]:
def get_rule_6(baoyangdf):
    rule4=baoyangdf[baoyangdf['人群']=='有首保,2次常规,预测第3次常规']
    rule4=get_last_run(rule4) 
    
    print("注意,此处的baoyang_id是倒序!!!")
    rule4['baoyang_id']=rule4.groupby('VIN')['修理日期'].rank(method='dense',ascending=False)
    rule4=rule4[rule4['baoyang_id']<=2]
#     print(rule4.shape)
#     print(rule4['VIN'].nunique())
#     print(rule4[['VIN','修理日期','修理日期_last','baoyang_id']].head(10))
    
    rule4.loc[rule4['公里数差']<=rule4['公里数差'].quantile(0.02),'公里数差']=rule4['公里数差'].quantile(0.02)
    rule4.loc[rule4['公里数差']>=rule4['公里数差'].quantile(0.98),'公里数差']=rule4['公里数差'].quantile(0.98)
    
#     #因为进行了天数差trick,因此没必要平滑
#     rule4.loc[rule4['天数差']<=rule4['天数差'].quantile(0.002),'天数差']=rule4['天数差'].quantile(0.002)
#     rule4.loc[rule4['天数差']>=rule4['天数差'].quantile(0.95),'天数差']=rule4['天数差'].quantile(0.95)

    rule4['日均']=rule4['公里数差']/rule4['天数差']
    rule4['baoyang_id']=rule4['baoyang_id'].astype(int).astype(str)
    rule44=pd.pivot(rule4,index='VIN',columns='baoyang_id',values=['修理日期','公里数','天数差','公里数差','日均','经销商代码'])
    rule44=rule44.reset_index()
    rule44.columns=[''.join(x) for x in rule44.columns.tolist()]

    rule44['mile_mean']=rule44['公里数差1']+rule44['公里数差2']
    rule44['mile_mean']=rule44['mile_mean']/2
    rule44['mile_sub']=(rule44['公里数差1']-rule44['公里数差2']).apply(abs)
    rule44['mile_per']=rule44['mile_sub']/(rule44['mile_mean'])

    rule44['day_mean']=rule44['天数差1']+rule44['天数差2']
    rule44['day_mean']=rule44['day_mean']/2
    rule44['day_sub']=(rule44['天数差1']-rule44['天数差2']).apply(abs)
    rule44['day_per']=rule44['day_sub']/(rule44['day_mean'])
    
    rule44['dayofmile_mean']=rule44['日均1']+rule44['日均2']
    rule44['dayofmile_mean']=rule44['dayofmile_mean']/2
    rule44['dayofmile_sub']=(rule44['日均1']-rule44['日均2']).apply(abs)
    rule44['dayofmile_per']=rule44['dayofmile_sub']/(rule44['dayofmile_mean'])

    rule44.loc[(rule44['mile_per']<=0.1),'milediff_pred']=rule44[rule44['mile_per']<=0.1][['公里数差2','公里数差1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.1)&(rule44['mile_per']<=0.2),'milediff_pred']=rule44[(rule44['mile_per']>0.1)&((rule44['mile_per']<=0.2))][['公里数差2','公里数差1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.2)&(rule44['mile_per']<=0.3),'milediff_pred']=rule44[(rule44['mile_per']>0.2)&((rule44['mile_per']<=0.3))][['公里数差2','公里数差1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.3)&(rule44['mile_per']<=0.4),'milediff_pred']=rule44[(rule44['mile_per']>0.3)&((rule44['mile_per']<=0.4))][['公里数差2','公里数差1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.4)&(rule44['mile_per']<=0.5),'milediff_pred']=rule44[(rule44['mile_per']>0.4)&((rule44['mile_per']<=0.5))][['公里数差2','公里数差1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.5)&(rule44['mile_per']<=0.6),'milediff_pred']=rule44[(rule44['mile_per']>0.5)&((rule44['mile_per']<=0.6))][['公里数差2','公里数差1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.6)&(rule44['mile_per']<=0.7),'milediff_pred']=rule44[(rule44['mile_per']>0.6)&((rule44['mile_per']<=0.7))][['公里数差2','公里数差1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.7)&(rule44['mile_per']<=0.8),'milediff_pred']=rule44[(rule44['mile_per']>0.7)&((rule44['mile_per']<=0.8))][['公里数差2','公里数差1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.8)&(rule44['mile_per']<=0.9),'milediff_pred']=rule44[(rule44['mile_per']>0.8)&((rule44['mile_per']<=0.9))][['公里数差2','公里数差1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.9),'milediff_pred']=rule44[rule44['mile_per']>0.9][['公里数差2','公里数差1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44.loc[(rule44['day_per']<=0.1),'daydiff_pred']=rule44[rule44['day_per']<=0.1][['天数差2','天数差1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.1)&(rule44['day_per']<=0.2),'daydiff_pred']=rule44[(rule44['day_per']>0.1)&((rule44['day_per']<=0.2))][['天数差2','天数差1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.2)&(rule44['day_per']<=0.3),'daydiff_pred']=rule44[(rule44['day_per']>0.2)&((rule44['day_per']<=0.3))][['天数差2','天数差1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.3)&(rule44['day_per']<=0.4),'daydiff_pred']=rule44[(rule44['day_per']>0.3)&((rule44['day_per']<=0.4))][['天数差2','天数差1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.4)&(rule44['day_per']<=0.5),'daydiff_pred']=rule44[(rule44['day_per']>0.4)&((rule44['day_per']<=0.5))][['天数差2','天数差1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.5)&(rule44['day_per']<=0.6),'daydiff_pred']=rule44[(rule44['day_per']>0.5)&((rule44['day_per']<=0.6))][['天数差2','天数差1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.6)&(rule44['day_per']<=0.7),'daydiff_pred']=rule44[(rule44['day_per']>0.6)&((rule44['day_per']<=0.7))][['天数差2','天数差1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.7)&(rule44['day_per']<=0.8),'daydiff_pred']=rule44[(rule44['day_per']>0.7)&((rule44['day_per']<=0.8))][['天数差2','天数差1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.8)&(rule44['day_per']<=0.9),'daydiff_pred']=rule44[(rule44['day_per']>0.8)&((rule44['day_per']<=0.9))][['天数差2','天数差1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.9),'daydiff_pred']=rule44[rule44['day_per']>0.9][['天数差2','天数差1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44.loc[(rule44['dayofmile_per']<=0.1),'dayofmile_pred']=rule44[rule44['dayofmile_per']<=0.1][['日均2','日均1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.1)&(rule44['dayofmile_per']<=0.2),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.1)&((rule44['dayofmile_per']<=0.2))][['日均2','日均1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.2)&(rule44['dayofmile_per']<=0.3),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.2)&((rule44['dayofmile_per']<=0.3))][['日均2','日均1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.3)&(rule44['dayofmile_per']<=0.4),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.3)&((rule44['dayofmile_per']<=0.4))][['日均2','日均1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.4)&(rule44['dayofmile_per']<=0.5),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.4)&((rule44['dayofmile_per']<=0.5))][['日均2','日均1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.5)&(rule44['dayofmile_per']<=0.6),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.5)&((rule44['dayofmile_per']<=0.6))][['日均2','日均1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.6)&(rule44['dayofmile_per']<=0.7),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.6)&((rule44['dayofmile_per']<=0.7))][['日均2','日均1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.7)&(rule44['dayofmile_per']<=0.8),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.7)&((rule44['dayofmile_per']<=0.8))][['日均2','日均1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.8)&(rule44['dayofmile_per']<=0.9),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.8)&((rule44['dayofmile_per']<=0.9))][['日均2','日均1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.9),'dayofmile_pred']=rule44[rule44['dayofmile_per']>0.9][['日均2','日均1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44['daydiff_pred']=rule44['daydiff_pred'].apply(lambda x:math.ceil(x))
#     rule44['next_date']=rule44[['修理日期1','daydiff_pred']].apply(lambda row:row[0]+relativedelta(days=row[1]),raw=True,axis=1)

    rule44['milediff_pred']=rule44['milediff_pred'].apply(lambda x:math.ceil(x))
#     rule44['next_mile']=rule44['公里数1']+rule44['milediff_pred']
        
    rule44['dayofmile_pred']=rule44['dayofmile_pred'].apply(float)
    
    print("检查是否有缺失值")
    print(pd.isnull(rule44[['VIN','daydiff_pred','milediff_pred','dayofmile_pred']]).sum())
    
    return rule44

def get_final_rule_6(baoyangdf1,x,y,veh2):
    rule_name='有首保,2次常规,预测第3次常规'
    batch=6
    
    end_id=1
    print("######提取预测集#######")
    rule111=get_rule_6(baoyangdf1)
    
    print('##########搜索最佳参数#######')
    validf=get_rule_6(x)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [38]:
_=get_final_rule_6(baoyangdf1,x,y,veh2)

######提取预测集#######
注意,此处的baoyang_id是倒序!!!
检查是否有缺失值
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
##########搜索最佳参数#######
注意,此处的baoyang_id是倒序!!!
检查是否有缺失值
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
##########train结果#########
    天数偏差人群     人数        占比  idx
0    0-10天   2643  0.093339    0
1   10-20天   2345  0.082815    1
2   20-30天   2201  0.077730    2
3   30-40天   1974  0.069713    3
4   40-50天   1767  0.062403    4
5   50-60天   1619  0.057176    5
6   60-70天   1401  0.049477    6
7   70-80天   1289  0.045522    7
8   80-90天   1066  0.037647    8
9  90-inf天  12011  0.424177    9
       公里数偏差人群    人数        占比  idx
0     0-1000公里  8407  0.296899    0
1  1000-2000公里  5647  0.199428    1
2  2000-3000公里  3478  0.122828    2
3  3000-4000公里  2101  0.074198    3
4  4000-5000公里  1477  0.052161    4
5  5000-6000公里  1084  0.038282    5
6  6000-7000公里   858  0.030301    6
7  7000-8000公里   690  0.02436

#### <font color=red>规则人群7:3次及以上常规,预测接下去常规

In [39]:
def get_rule_7(baoyangdf):
    rule4=baoyangdf[baoyangdf['人群']=='3次及以上常规,预测接下去常规']
    rule4=get_last_run(rule4) 
    
    print("注意,此处的baoyang_id是倒序!!!")
    rule4['baoyang_id']=rule4.groupby('VIN')['修理日期'].rank(method='dense',ascending=False)
    rule4=rule4[rule4['baoyang_id']<=2]
#     print(rule4.shape)
#     print(rule4['VIN'].nunique())
#     print(rule4[['VIN','修理日期','修理日期_last','baoyang_id']].head(10))
    
    rule4.loc[rule4['公里数差']<=rule4['公里数差'].quantile(0.02),'公里数差']=rule4['公里数差'].quantile(0.02)
    rule4.loc[rule4['公里数差']>=rule4['公里数差'].quantile(0.98),'公里数差']=rule4['公里数差'].quantile(0.98)
    
#     #因为进行了天数差trick,因此没必要平滑
#     rule4.loc[rule4['天数差']<=rule4['天数差'].quantile(0.002),'天数差']=rule4['天数差'].quantile(0.002)
#     rule4.loc[rule4['天数差']>=rule4['天数差'].quantile(0.95),'天数差']=rule4['天数差'].quantile(0.95)

    rule4['日均']=rule4['公里数差']/rule4['天数差']
    rule4['baoyang_id']=rule4['baoyang_id'].astype(int).astype(str)
    rule44=pd.pivot(rule4,index='VIN',columns='baoyang_id',values=['修理日期','公里数','天数差','公里数差','日均','经销商代码'])
    rule44=rule44.reset_index()
    rule44.columns=[''.join(x) for x in rule44.columns.tolist()]

    rule44['mile_mean']=rule44['公里数差1']+rule44['公里数差2']
    rule44['mile_mean']=rule44['mile_mean']/2
    rule44['mile_sub']=(rule44['公里数差1']-rule44['公里数差2']).apply(abs)
    rule44['mile_per']=rule44['mile_sub']/(rule44['mile_mean'])

    rule44['day_mean']=rule44['天数差1']+rule44['天数差2']
    rule44['day_mean']=rule44['day_mean']/2
    rule44['day_sub']=(rule44['天数差1']-rule44['天数差2']).apply(abs)
    rule44['day_per']=rule44['day_sub']/(rule44['day_mean'])
    
    rule44['dayofmile_mean']=rule44['日均1']+rule44['日均2']
    rule44['dayofmile_mean']=rule44['dayofmile_mean']/2
    rule44['dayofmile_sub']=(rule44['日均1']-rule44['日均2']).apply(abs)
    rule44['dayofmile_per']=rule44['dayofmile_sub']/(rule44['dayofmile_mean'])

    rule44.loc[(rule44['mile_per']<=0.1),'milediff_pred']=rule44[rule44['mile_per']<=0.1][['公里数差2','公里数差1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.1)&(rule44['mile_per']<=0.2),'milediff_pred']=rule44[(rule44['mile_per']>0.1)&((rule44['mile_per']<=0.2))][['公里数差2','公里数差1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.2)&(rule44['mile_per']<=0.3),'milediff_pred']=rule44[(rule44['mile_per']>0.2)&((rule44['mile_per']<=0.3))][['公里数差2','公里数差1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.3)&(rule44['mile_per']<=0.4),'milediff_pred']=rule44[(rule44['mile_per']>0.3)&((rule44['mile_per']<=0.4))][['公里数差2','公里数差1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.4)&(rule44['mile_per']<=0.5),'milediff_pred']=rule44[(rule44['mile_per']>0.4)&((rule44['mile_per']<=0.5))][['公里数差2','公里数差1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.5)&(rule44['mile_per']<=0.6),'milediff_pred']=rule44[(rule44['mile_per']>0.5)&((rule44['mile_per']<=0.6))][['公里数差2','公里数差1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.6)&(rule44['mile_per']<=0.7),'milediff_pred']=rule44[(rule44['mile_per']>0.6)&((rule44['mile_per']<=0.7))][['公里数差2','公里数差1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.7)&(rule44['mile_per']<=0.8),'milediff_pred']=rule44[(rule44['mile_per']>0.7)&((rule44['mile_per']<=0.8))][['公里数差2','公里数差1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.8)&(rule44['mile_per']<=0.9),'milediff_pred']=rule44[(rule44['mile_per']>0.8)&((rule44['mile_per']<=0.9))][['公里数差2','公里数差1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['mile_per']>0.9),'milediff_pred']=rule44[rule44['mile_per']>0.9][['公里数差2','公里数差1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44.loc[(rule44['day_per']<=0.1),'daydiff_pred']=rule44[rule44['day_per']<=0.1][['天数差2','天数差1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.1)&(rule44['day_per']<=0.2),'daydiff_pred']=rule44[(rule44['day_per']>0.1)&((rule44['day_per']<=0.2))][['天数差2','天数差1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.2)&(rule44['day_per']<=0.3),'daydiff_pred']=rule44[(rule44['day_per']>0.2)&((rule44['day_per']<=0.3))][['天数差2','天数差1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.3)&(rule44['day_per']<=0.4),'daydiff_pred']=rule44[(rule44['day_per']>0.3)&((rule44['day_per']<=0.4))][['天数差2','天数差1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.4)&(rule44['day_per']<=0.5),'daydiff_pred']=rule44[(rule44['day_per']>0.4)&((rule44['day_per']<=0.5))][['天数差2','天数差1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.5)&(rule44['day_per']<=0.6),'daydiff_pred']=rule44[(rule44['day_per']>0.5)&((rule44['day_per']<=0.6))][['天数差2','天数差1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.6)&(rule44['day_per']<=0.7),'daydiff_pred']=rule44[(rule44['day_per']>0.6)&((rule44['day_per']<=0.7))][['天数差2','天数差1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.7)&(rule44['day_per']<=0.8),'daydiff_pred']=rule44[(rule44['day_per']>0.7)&((rule44['day_per']<=0.8))][['天数差2','天数差1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.8)&(rule44['day_per']<=0.9),'daydiff_pred']=rule44[(rule44['day_per']>0.8)&((rule44['day_per']<=0.9))][['天数差2','天数差1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['day_per']>0.9),'daydiff_pred']=rule44[rule44['day_per']>0.9][['天数差2','天数差1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44.loc[(rule44['dayofmile_per']<=0.1),'dayofmile_pred']=rule44[rule44['dayofmile_per']<=0.1][['日均2','日均1']].apply(lambda row:0.45*row[0]+0.55*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.1)&(rule44['dayofmile_per']<=0.2),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.1)&((rule44['dayofmile_per']<=0.2))][['日均2','日均1']].apply(lambda row:0.4*row[0]+0.6*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.2)&(rule44['dayofmile_per']<=0.3),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.2)&((rule44['dayofmile_per']<=0.3))][['日均2','日均1']].apply(lambda row:0.35*row[0]+0.65*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.3)&(rule44['dayofmile_per']<=0.4),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.3)&((rule44['dayofmile_per']<=0.4))][['日均2','日均1']].apply(lambda row:0.3*row[0]+0.7*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.4)&(rule44['dayofmile_per']<=0.5),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.4)&((rule44['dayofmile_per']<=0.5))][['日均2','日均1']].apply(lambda row:0.25*row[0]+0.75*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.5)&(rule44['dayofmile_per']<=0.6),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.5)&((rule44['dayofmile_per']<=0.6))][['日均2','日均1']].apply(lambda row:0.2*row[0]+0.8*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.6)&(rule44['dayofmile_per']<=0.7),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.6)&((rule44['dayofmile_per']<=0.7))][['日均2','日均1']].apply(lambda row:0.15*row[0]+0.85*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.7)&(rule44['dayofmile_per']<=0.8),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.7)&((rule44['dayofmile_per']<=0.8))][['日均2','日均1']].apply(lambda row:0.1*row[0]+0.9*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.8)&(rule44['dayofmile_per']<=0.9),'dayofmile_pred']=rule44[(rule44['dayofmile_per']>0.8)&((rule44['dayofmile_per']<=0.9))][['日均2','日均1']].apply(lambda row:0.05*row[0]+0.95*row[1],raw=True,axis=1)
    rule44.loc[(rule44['dayofmile_per']>0.9),'dayofmile_pred']=rule44[rule44['dayofmile_per']>0.9][['日均2','日均1']].apply(lambda row:1*row[1],raw=True,axis=1)

    rule44['daydiff_pred']=rule44['daydiff_pred'].apply(lambda x:math.ceil(x))
#     rule44['next_date']=rule44[['修理日期1','daydiff_pred']].apply(lambda row:row[0]+relativedelta(days=row[1]),raw=True,axis=1)

    rule44['milediff_pred']=rule44['milediff_pred'].apply(lambda x:math.ceil(x))
#     rule44['next_mile']=rule44['公里数1']+rule44['milediff_pred']
        
    rule44['dayofmile_pred']=rule44['dayofmile_pred'].apply(float)
    
    print("检查是否有缺失值")
    print(pd.isnull(rule44[['VIN','daydiff_pred','milediff_pred','dayofmile_pred']]).sum())
    
    return rule44

def get_final_rule_7(baoyangdf1,x,y,veh2):
    rule_name='3次及以上常规,预测接下去常规'
    batch=7
    
    end_id=1
    print("######提取预测集#######")
    rule111=get_rule_7(baoyangdf1)
    
    print('##########搜索最佳参数#######')
    validf=get_rule_7(x)
    if 'last_baoyang_date' in list(validf.columns):
        del validf['last_baoyang_date']
    if 'last_baoyang_mile' in list(validf.columns):
        del validf['last_baoyang_mile']
    validf=validf.merge(y,on=['VIN'],how='left')
    
    print('##########train结果#########')
    validf_0=validf.copy()
    is_train='True'
    final_vali_1=get_next_day_dayofmile(validf_0,end_id,is_train)
    version='day+dayofmile'
    piancha_1=evaluate(final_vali_1,version,rule_name)
    eva_1=get_eva_score(final_vali_1,version,batch,rule_name)

    final_vali_2=get_next_mile_dayofmile(validf_0,end_id,is_train)
    version='mile+dayofmile'
    piancha_2=evaluate(final_vali_2,version,rule_name)
    eva_2=get_eva_score(final_vali_2,version,batch,rule_name)
    
    final_vali_3=get_next_day_mile(validf_0,end_id,is_train)
    version='day+mile'
    final_vali_3['最后一次保养日期']=final_vali_3['修理日期%d'%end_id]
    final_vali_3['最后一次保养公里数']=final_vali_3['公里数%d'%end_id]
    final_vali_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark','daydiff_true','milediff_true','dayofmile_true',
          'last_baoyang_date', 'last_baoyang_mile']
    final_vali_3=final_vali_3[cols].drop_duplicates() 
    final_vali_3.to_pickle(savep+'%d_rule_train.pkl'%batch)
    piancha_3=evaluate(final_vali_3,version,rule_name)
    eva_3=get_eva_score(final_vali_3,version,batch,rule_name)
    
    piancha=pd.concat([piancha_1,piancha_2,piancha_3],axis=0)
    piancha.to_pickle(savep+'%d_rule_偏差评估.pkl'%batch)
    
    eva=pd.concat([eva_1,eva_2,eva_3],axis=0)
    eva.to_pickle(savep+'%d_rule_规则训练评价指标.pkl'%batch)
    
    print('##########test结果#########')
    rule111_0=rule111.copy()
    is_train='False'
    final_rule_3=get_next_day_mile(rule111_0,end_id,is_train)
    final_rule_3['最后一次保养日期']=final_rule_3['修理日期%d'%end_id]
    final_rule_3['最后一次保养公里数']=final_rule_3['公里数%d'%end_id]
    final_rule_3['mark']=rule_name
    cols=['VIN','最后一次保养日期','最后一次保养公里数','daydiff_pred','milediff_pred',
          'dayofmile_pred','next_date','next_mile','mark']
    final_rule_3=final_rule_3[cols].drop_duplicates()   
    final_rule_3.to_pickle(savep+'%d_rule_test.pkl'%batch)

    return []

In [40]:
_=get_final_rule_7(baoyangdf1,x,y,veh2)

######提取预测集#######
注意,此处的baoyang_id是倒序!!!
检查是否有缺失值
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
##########搜索最佳参数#######
注意,此处的baoyang_id是倒序!!!
检查是否有缺失值
VIN               0
daydiff_pred      0
milediff_pred     0
dayofmile_pred    0
dtype: int64
##########train结果#########
    天数偏差人群     人数        占比  idx
0    0-10天  13119  0.108575    0
1   10-20天  11205  0.092734    1
2   20-30天  10049  0.083167    2
3   30-40天   8831  0.073087    3
4   40-50天   7602  0.062915    4
5   50-60天   6552  0.054225    5
6   60-70天   5771  0.047762    6
7   70-80天   5095  0.042167    7
8   80-90天   4390  0.036332    8
9  90-inf天  48215  0.399035    9
       公里数偏差人群     人数        占比  idx
0     0-1000公里  35724  0.295657    0
1  1000-2000公里  22264  0.184260    1
2  2000-3000公里  13553  0.112167    2
3  3000-4000公里   8830  0.073078    3
4  4000-5000公里   6316  0.052272    4
5  5000-6000公里   4684  0.038766    5
6  6000-7000公里   3843  0.031805    6
7  7000-8000公里   3162

### <font color=red>合并

In [41]:
# rule_train1=pd.concat([val0,val2,val3,val4,val5,val6,val7],axis=0)
# rule_train1=rule_train1.merge(y,on=['VIN'],how='left')
# rule_train1.to_pickle(report_savep+'rule_train.pkl')
# print(rule_train1.shape)
# print(rule_train1['VIN'].nunique())

In [42]:
# rule_result1=pd.concat([fin0,fin1,fin2,fin3,fin4,fin5,fin6,fin7],axis=0)
# rule_result1.to_pickle(report_savep+'rule_test.pkl')
# print(rule_result1.shape)
# print(rule_result1['VIN'].nunique())

In [43]:
print((time.time()-s1)/60)

8.71423538128535
